In [1]:
from PIL import Image
import glob
import numpy as np
import os
from os.path import basename

image_Lbl_list = []
for filename in glob.glob('Dataset/*.png'): #assuming png
    im = Image.open(filename)
    label = (filename.split("_")[1])
    #print(label)
    image_Lbl_list.append(label)
    im.close()

#print(image_Lbl_list)

In [2]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

flat_Image_List = [] # new list containing flatten image arrays
#for img in image_Lbl_list: # iterate over the images contained in the directory
for filename in glob.glob('Dataset/*.png'):
    im = Image.open(filename)
    img = np.reshape(im, (12288))
    flat_Image_List.append(img) # Append to the list 
    #plt.imshow(img)
    im.close()
    #print(flat_Image_List)

#print(flat_Image_List)

In [3]:
labels = 15
imageSize = 12288

In [4]:
from numpy import array

xTrain = array(flat_Image_List)

#print(flat_Image_List)
print(xTrain)

[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]


In [5]:
# Converts RGB to a range between 0-1
xTrain = xTrain/255

In [15]:
xTrain = xTrain.reshape(xTrain.shape[0], 64, 64, 1)

ValueError: cannot reshape array of size 897699840 into shape (73055,64,64,1)

In [10]:
input_shape = (64, 64, 1)

In [12]:
xTrain = xTrain.astype('float32')

In [13]:
# For encoding categorical variables and pre processing.
import sklearn.preprocessing as pre
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

encoder = pre.LabelBinarizer()
encoder.fit(image_Lbl_list)
outputs = encoder.transform(image_Lbl_list)

#xTrain.reshape(100, 12288)

inputs = xTrain
#inputs.reshape(len(xTrain), imageSize)
inputs.shape

(73055, 12288)

In [19]:
# ------- MODEL -------
# Import keras.
import keras as kr
import tensorflow as tf

# Importing the required Keras modules containing model and layers
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D

# Start a neural network, building it by layers.
model = kr.models.Sequential()
# Add a hidden layer with 784 neurons.
model.add(Conv2D(64, kernel_size=(3,3), input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten()) # Flattening the 2D arrays for fully connected layers
model.add(Dense(128, activation=tf.nn.relu))
model.add(Dropout(0.2))
model.add(Dense(10,activation=tf.nn.softmax))

# Add a 15 neuron output layer.
model.add(kr.layers.Dense(units=15, activation='softmax'))

In [20]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 62, 62, 64)        640       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 31, 31, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 61504)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               7872640   
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 10)                1290      
_________________________________________________________________
dense_7 (Dense)              (None, 15)                165       
Total para

In [21]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Number of Epoch is the amount of times the training set is put through the model
# The batch size is the amount of images the models processes at one time
model.fit(x=inputs,y=outputs, epochs=10, batch_size=120)

ValueError: Error when checking input: expected conv2d_3_input to have 4 dimensions, but got array with shape (73055, 12288)